In [1]:

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import root_mean_squared_error
from src.algorithms.random_forest import train_and_test_random_forest
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score



In [ ]:
from entsoe import EntsoePandasClient
api_key = pd.read_csv('data/api_key.csv').iloc[0, 0]
client = EntsoePandasClient(api_key=api_key)

In [ ]:
from src.entsoe_data_imports import (
    get_generation,
    get_wind_solar_forecast,
    get_load_and_forecast,
    get_capacities,
)

years = [2022, 2023, 2024]
zones = ['NO_2', 'NO_3', 'NO_4', 'DK_1', 'DK_2']
download_data = True
if download_data:
    get_generation(client, zones, years)
    get_wind_solar_forecast(client, zones, years)
    get_load_and_forecast(client, zones, years)
    get_capacities(client, zones, years, carrier='Wind Onshore')
    


In [ ]:
from src.analysis import get_fitting_params
from src.algorithms import train_and_test_lstm
from src.load_saved_data import (
    load_load_data,
    load_wind_data,
)


def asymptotic_func(x, a, b):
    return a * x / (x + b)

error_type = "load"
zone = 'NO_2'
years = [2022, 2023, 2024]

if error_type == "wind":
    target_series, forecast_series, scaling_value = load_wind_data(years, zone)
else:
    target_series, forecast_series, scaling_value = load_load_data(years, zone)


max_lead_time = 12
for error_type in ["wind"]:
    if error_type == "wind":
        zones = ['NO_2', 'NO_3', 'NO_4', 'DK_1', 'DK_2']
    else:
        zones = ['NO_1', 'NO_2', 'NO_3', 'NO_4', 'NO_5', 'DK_1']
        # zones = ['NO_2']
    param_df = get_fitting_params(
        error_type, 
        years, 
        zones, 
        train_and_test_function=train_and_test_lstm, 
        fitting_function=asymptotic_func,
        n_lags=3, 
        max_lead_time=max_lead_time
        )
    param_df.to_csv(f"data/processed/{error_type}_fitting_params_lead_time_{max_lead_time}.csv")





In [ ]:
plt.plot(target_series.index[:1000], target_series.values[:1000], label='Detrended Target')
plt.plot(forecast_series.index[:1000], forecast_series.values[:1000], label='Detrended Forecast', linestyle='--', linewidth=0.5)


In [ ]:


from src.load_saved_data import (
    load_load_data,
    load_wind_data,
)
from src.algorithms.lstm import train_and_test_lstm


max_lead_time_dict = {
    "wind": 12,
    "load": 6
}
n_lags = 3
rmse_per_step = []

zone = 'NO_3'  # Example zone
rmse_dict = {}
# for error_type in ["wind", "load"]:
for error_type in ["wind", "load"]:
    if error_type == "wind":
        target_series, forecast_series, scaling_value = load_wind_data(years, zone)
    else:
        target_series, forecast_series, scaling_value = load_load_data(years, zone)

    rmse_per_step = []
    max_lead_time = max_lead_time_dict[error_type]
    for n_lead_time in range(1, max_lead_time + 1):                 
        # y_test, y_pred = train_and_test_random_forest(target_series, forecast_series, n_lags=n_lags, n_lead_time=n_lead_time)
        y_test, y_pred = train_and_test_lstm(target_series, forecast_series, target_series - forecast_series, target_series.index.hour, n_lags=n_lags, n_lead_time=n_lead_time)
        
        rmse = root_mean_squared_error(y_test, y_pred) / scaling_value
        rmse_per_step.append(rmse)


    rmse_dict[error_type] = rmse_per_step
# Assume rmse_per_step is already defined



In [ ]:
plt.figure(figsize=(6, 4))

error_colors = {
    "wind": "tab:green",  
    "load": "darkred",  
}

for error_type, rmse_per_step in rmse_dict.items():
    horizons = np.arange(1, len(rmse_per_step) + 1)
    error_type_nice_name = "Wind" if error_type == "wind" else "Load"
    horizons = np.arange(1, max_lead_time_dict[error_type] + 1)
    # Ensure rmse_per_step is a numpy array
    rmse_per_step = np.array(rmse_per_step)
    plotting_linspace = np.linspace(0, 12, 100)
    # Fit the function to the data

    asymptotic_params, _ = curve_fit(asymptotic_func, horizons, rmse_per_step)
    asymptotic_fitted_rmse = asymptotic_func(plotting_linspace, *asymptotic_params)

    plt.scatter(horizons, 100 * rmse_per_step, color=error_colors[error_type], marker='x', label=f'{error_type_nice_name}', s=100)
    plt.plot(plotting_linspace, 100 * asymptotic_fitted_rmse, linestyle='--', color=error_colors[error_type], label=rf'Fit {error_type}')

plt.xlabel("Lead time [h]")
plt.xticks([0, 3, 6, 9, 12])

plt.ylabel("NRMSE [%]")
plt.grid(True)
plt.legend(bbox_to_anchor=(0., 1.02), loc='lower left', borderaxespad=0., ncols=2)
plt.tight_layout()
plt.xlim((0, 12))
plt.savefig(f'plots/nrmse_lag_{n_lags}_{zone}_top_legend.pdf', bbox_inches='tight')
plt.show()